### Importamos las librerias

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Ejecutamos el Scraping

In [2]:
# URL de la API
url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.geojson"

# Parámetros de la solicitud GET
params = {
    "api_key": "DEMO_KEY",
    "fuel_type": "ELEC",
    "state": "NY",
        
}

# Realizar la solicitud GET
response = requests.get(url, params=params)

# Verificar el estado de la solicitud
if response.status_code == 200:
    # Analizar el contenido JSON de la respuesta
    data = response.json()

    # Extraer la información relevante del JSON
    features = data['features']
    
    # Crear una lista para almacenar los datos
    rows = []
    for feature in features:
        properties = feature['properties']
        station_id = properties['id']
        station_name = properties['station_name']
        latitude = feature['geometry']['coordinates'][1]
        longitude = feature['geometry']['coordinates'][0]
        occupancy = properties['maximum_vehicle_class']
        city = properties['city']
        network = properties['ev_network']
        connector_types = properties['ev_connector_types']
        owner = properties['owner_type_code']
        facility_type = properties['facility_type']
        rows.append([station_id, station_name, latitude, longitude, occupancy, city, network,connector_types, owner, facility_type ])

    # Crear un DataFrame usando pandas
    columns = ['estacionID', 'nombre', 'latitud', 'longitud', 'ocupacion_maxima', 'Zone', 'red', 'tipo_conector', 'dueño', 'tipo_estacion']
    df = pd.DataFrame(rows, columns=columns)

    # Imprimir el DataFrame
    print(df)
else:
    print("Error al realizar la solicitud:", response.status_code)

      estacionID                                             nombre  \
0          35272  New York Institute of Technology - Old Westbur...   
1          35562  New York State Energy Research Development Aut...   
2          35563                  Saratoga Technology & Energy Park   
3          36532  Hudson Valley Community College - TEC-SMART Bu...   
4          41240                   James J Peters VA Medical Center   
...          ...                                                ...   
4350      331865                             PANYNJ LGATCGARAGE 5-3   
4351      331866                             PANYNJ LGATCGARAGE 5-4   
4352      331867                             PANYNJ LGATCGARAGE 5-5   
4353      331868                             PANYNJ LGATCGARAGE 5-2   
4354      331871                          NYC FLEET DPRWFMARNA_1_L3   

        latitud   longitud ocupacion_maxima          Zone  \
0     40.814019 -73.608978               LD  Old Westbury   
1     42.702424 -73.85769

### corremos el data frame

In [3]:
df

,estacionID,nombre,latitud,longitud,ocupacion_maxima,Zone,red,tipo_conector,dueño,tipo_estacion
0,35272,New York Institute of Technology - Old Westbur...,40.814019,-73.608978,LD,Old Westbury,Non-Networked,[NEMA520],P,COLLEGE_CAMPUS
1,35562,New York State Energy Research Development Aut...,42.702424,-73.857693,LD,Albany,Non-Networked,"[J1772, NEMA520]",P,STATE_GOV
2,35563,Saratoga Technology & Energy Park,42.969823,-73.773021,LD,Malta,Non-Networked,[NEMA520],SG,RESEARCH_FACILITY
3,36532,Hudson Valley Community College - TEC-SMART Bu...,42.971670,-73.774182,LD,Malta,Non-Networked,[NEMA520],SG,COLLEGE_CAMPUS
4,41240,James J Peters VA Medical Center,40.867403,-73.906405,None,Bronx,Non-Networked,[J1772],FG,HOSPITAL
...,...,...,...,...,...,...,...,...,...,...
4350,331865,PANYNJ LGATCGARAGE 5-3,40.769509,-73.865421,None,Queens,ChargePoint Network,[J1772],None,None
4351,331866,PANYNJ LGATCGARAGE 5-4,40.769470,-73.865591,None,Queens,ChargePoint Network,[J1772],None,None
4352,331867,PANYNJ LGATCGARAGE 5-5,40.769452,-73.865500,None,Queens,ChargePoint Network,[J1772],None,None
4353,331868,PANYNJ LGATCGARAGE 5-2,40.769481,-73.865372,None,Queens,ChargePoint Network,[J1772],None,None


#### Modificamos el nombre Tesla por Tesla Supercharger para tener concordancia con el diccionario del dataset

In [4]:
df.loc[df['red'] == 'Tesla', 'red'] = 'Tesla Supercharger'

Generamos la tabla de conectores

In [6]:
data_it1 = df.explode(['tipo_conector'])
data_it2 = pd.json_normalize(data_it1['tipo_conector']).set_index(data_it1['tipo_conector'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)

In [7]:
data_it3 = data_it3.drop_duplicates()

In [8]:
data_it3

,estacionID,nombre,latitud,longitud,ocupacion_maxima,Zone,red,tipo_conector,dueño,tipo_estacion
0,35272,New York Institute of Technology - Old Westbur...,40.814019,-73.608978,LD,Old Westbury,Non-Networked,NEMA520,P,COLLEGE_CAMPUS
1,35562,New York State Energy Research Development Aut...,42.702424,-73.857693,LD,Albany,Non-Networked,J1772,P,STATE_GOV
1,35562,New York State Energy Research Development Aut...,42.702424,-73.857693,LD,Albany,Non-Networked,NEMA520,P,STATE_GOV
2,35563,Saratoga Technology & Energy Park,42.969823,-73.773021,LD,Malta,Non-Networked,NEMA520,SG,RESEARCH_FACILITY
3,36532,Hudson Valley Community College - TEC-SMART Bu...,42.971670,-73.774182,LD,Malta,Non-Networked,NEMA520,SG,COLLEGE_CAMPUS
...,...,...,...,...,...,...,...,...,...,...
4351,331866,PANYNJ LGATCGARAGE 5-4,40.769470,-73.865591,None,Queens,ChargePoint Network,J1772,None,None
4352,331867,PANYNJ LGATCGARAGE 5-5,40.769452,-73.865500,None,Queens,ChargePoint Network,J1772,None,None
4353,331868,PANYNJ LGATCGARAGE 5-2,40.769481,-73.865372,None,Queens,ChargePoint Network,J1772,None,None
4354,331871,NYC FLEET DPRWFMARNA_1_L3,40.760611,-73.856970,None,Queens,ChargePoint Network,CHADEMO,None,None


### Guardamos los datasets

In [ ]:
df.to_parquet('electChargers.parquet', index=False)

In [ ]:
data_it3.to_parquet('puenteCarga.parquet', index=False)